# Overview

Model-Based Temporal Abstraction involves simultaneuously learning
1) skill-conditioned low-level policy
2) skill-conditioned temporally abstract world model

Notation
- skill-conditioned low-level policy: $\pi_{\theta}(a_t|s_t, z)$
    - $\theta$ are parameters
    - $a_t \in A$ is current action selected by agent
    - $s_t \in S$ is current state
    - $z \in Z$ is abstract skill variable that encodes a skill

- skill-conditioned temporally abstract world model (TAWM): $p_{\psi}(s'|s,z)$ (models distribution of states agent is in after skill $z$)
    - $\psi$ parameters
    - $z$ is current skill

Note: low-level policy and TAWM not trained on rewards, reward function is provided later for planning with the learned skills 





In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal, Independent, kl_divergence
import numpy as np
import minari


device = "cuda" if torch.cuda.is_available() else "cpu"

### Learning $\pi_{\theta}$ and $p_{\psi}$

Learning $\pi_{\theta}$ and $p_{\psi}$ requires treating skills as latent variables and optimizing the ELBO

$$
\mathcal{L}(\theta,\psi,\phi,\omega)
= \mathbb{E}_{\tau_T \sim \mathcal{D}}\!\left[
  \mathbb{E}_{q_\phi(z\,|\,\tau_T)}\!\left[
    \log \pi_\theta(\bar{a}\,|\,\bar{s}, z)
    + \log p_\psi(s_T \,|\, s_0, z)
  \right]
  - D_{\mathrm{KL}}\!\left(q_\phi(z\,|\,\tau_T)\,\|\,p_\omega(z\,|\,s_0)\right)
\right].
$$

where $\tau_T$ is a T-length subtrajectory sampled from the offline dataset $\mathcal{D}$, $\bar{s}$ and $\bar{a}$ are state and action sequences of $\tau_T$, $q_{\psi}$ is a posterior over $z$ given $\tau_T$, and $p_{\omega}$ is a prior of $z$ given $s_0$.

The first term is the log-likelihood of demonstrator actions. This ensures that the low-level policy can reproduce a demonstrator's action sequence given a skill. This forces $z$ to encode control-relevant information.

The second term is the log-likelihood of long-term state transitions. This term ensures that we learn relationships between $z$ to what possible $s_T$ could result from. the skill.

Finally, the last term is the KL divergence between skill posterior and prior (encourages compression of skills). Therefore, maximizing this ELBO makes skills $z$ explain the data and keeps the KL divergence small. This ensures that the skill is start-state predictable.

In [41]:
NUM_NEURONS = 256
Z_DIM = 256

# Skill Posterior, q_phi
class SkillPosterior(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.fc1 = nn.Linear(in_features=self.state_dim, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
        self.bi_gru = nn.GRU(input_size=NUM_NEURONS+self.action_dim, hidden_size= NUM_NEURONS//2, bidirectional=True, batch_first=True)
        self.mean = MeanNetwork()
        self.std = StandardDeviationNetwork()

    def forward(self, state_sequence, action_sequence):
        embedded_states = self.relu(self.fc1(state_sequence))
        concatenated = torch.cat([embedded_states, action_sequence], dim=0)
        x, _ = self.bi_gru(concatenated)
        mean = MeanNetwork.forward(x)
        std = StandardDeviationNetwork.forward(x)
        return mean, std


# Low-Level Skill-Conditioned Policy, pi_theta
class SkillPolicy(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        
        self.state_dim = state_dim
        self.fc1 = nn.Linear(in_features=self.state_dim+Z_DIM, out_features=NUM_NEURONS)
        self.fc2 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
        self.mean = MeanNetwork()
        self.std = StandardDeviationNetwork()
    
    def forward(self, state, z):
        c = torch.cat([state, z], dim=0)
        x = self.relu(self.fc1(c))
        x = self.relu(self.fc2(x))
        mean = self.mean(x)
        std = self.std(x)
        return mean, std
        

# Temporally-Abstract World Model, p_psi
class TAWM(nn.Module):
    def __init__(self, state_dim):
        super().__init__()

        self.state_dim = state_dim
        self.fc1 = nn.Linear(in_features=self.state_dim+Z_DIM, out_features=NUM_NEURONS)
        self.fc2 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
        self.mean = MeanNetwork()
        self.std = StandardDeviationNetwork()
    
    def forward(self, input_state, z):
        c = torch.cat([input_state, z], dim=0)
        x = self.relu(self.fc1(c))
        x = self.relu(self.fc2(x))
        mean = self.mean(x)
        std = self.std(x)
        return mean, std

# Skill Prior, p_omega
class SkillPrior(nn.Module):
    def __init__(self, state_dim):
        super().__init__()

        self.state_dim = state_dim
        self.fc1 = nn.Linear(in_features=self.state_dim, out_features=NUM_NEURONS)
        self.fc2 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
        self.mean = MeanNetwork()
        self.std = StandardDeviationNetwork()
    
    def forward(self, input_state):
        x = self.relu(self.fc1(input_state))
        x = self.relu(self.fc2(x))
        mean = self.mean(x)
        std = self.std(x)
        return mean, std



class MeanNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.fc2 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
        
class StandardDeviationNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.fc2 = nn.Linear(in_features=NUM_NEURONS, out_features=NUM_NEURONS)
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softplus(x)
        return x


#### The Expectation-Maximization (EM) Algorithm

Since calculating the true posterior of $z$ given $\tau_T$ is intractable, we infer $q_{\psi}(z|\tau_T)$.

1. E-Step:
- Update $\psi$ w/gradient descent so that KL divergence between $q_\psi$ and true posterior is minimized

2. M-Step:
- Fixing $q_{\psi}$, update ($\theta, \psi, \omega$) s.t. ELBO is maximized using gradient ascent.


##### E-Step (Update $\psi$)

In this step, we want to minimize 

$$
\mathbb{E}_{\mathcal{T}_T\sim\mathcal{D}}
\Bigg[
\mathbb{E}_{z\sim q_\phi}
\bigg[
\log \frac{q_\phi\!\left(z\mid \bar{s},\bar{a}\right)}
{\pi_\theta\!\left(\bar{a}\mid \bar{s}, z\right)\,p_\omega\!\left(z\mid s_0\right)}
\bigg]
\Bigg]
$$

Equivalently, we want to minimize $\mathcal{KL}(q_{\psi}||p)$ where $p$, the true posterior is 

$$
p(z \mid \bar{s}, \bar{a}) = \frac{1}{\eta}\,\pi_\theta(\bar{a}\mid \bar{s}, z)\,p_\omega(z\mid s_0).
$$


##### M-Step (Update $\psi$)

In this step, we want to update $\theta$, $\psi$, and $\omega$ using gradient ascent to maximize the ELBO from above.

Both steps are trained using an Adam optimizer.


##### Dataset 1: AntMaze

In [55]:
ant_maze_dataset = minari.load_dataset('D4RL/antmaze/medium-play-v1')

In [56]:
print(ant_maze_dataset[0].actions.shape)
print(ant_maze_dataset[0].observations.keys())
print(ant_maze_dataset[0].observations["observation"].shape)

(1000, 8)
dict_keys(['achieved_goal', 'desired_goal', 'observation'])
(1001, 27)


In [42]:
# B, the number of subtrajectories per batch
B = 100

# T, the length of each subtrajectory
T = 10

# AntMaze state and action dims
state_dim = 27
action_dim = 8

q_phi = SkillPosterior(state_dim=state_dim, action_dim=action_dim).to(device)
pi_theta = SkillPolicy(state_dim=state_dim).to(device)
p_psi = TAWM(state_dim=state_dim).to(device)
p_omega = SkillPrior(state_dim=state_dim).to(device)

In [ ]:
# E loss
def compute_e_loss(subtrajectory_batch):
    zs = []
    for subtrajectory in subtrajectory_batch:
        states, actions = subtrajectory.actions, subtrajectory.observations["observation"]
        mu, std = q_phi(states, actions)
        epsilon = torch.randn(1) # epsilon ~ N(0, 1)
        z = mu + std * epsilon
        zs.append(z)

    e_loss = 0
    for i in range(B):
        states, actions =  subtrajectory_batch[i].actions, subtrajectory_batch[i].observations["observation"]
        mu_pi, std_pi = pi_theta(states, zs[i])
        pi_dist = Independent(Normal(mu_pi, std_pi), 1)
        log_pi_t = pi_dist.log_prob(actions)
        log_pi_seq = log_pi_t.sum(dim=1)

        mu_p_omega, std_p_omega = p_omega(states, zs[i])
        p_omega_dist = Independent(Normal(mu_p_omega, std_p_omega), 1)
        log_p_omega_t = p_omega_dist.log_prob(actions)
        log_p_omega_seq = log_p_omega_t.sum(dim=1)

        mu_q, std_q = q_phi(states, zs[i])
        q_dist = Independent(Normal(mu_q, std_q), 1)
        log_q_t = q_dist.log_prob(actions)
        log_q_seq = log_q_t.sum(dim=1)  

        e_loss += log_pi_seq + log_p_omega_seq - log_q_seq

    return e_loss * -(1/B)

    
# M loss
def compute_m_loss(subtrajectory_batch):
    zs = []
    for subtrajectory in subtrajectory_batch:
        states, actions = subtrajectory.actions, subtrajectory.observations["observation"]
        mu, std = q_phi(states, actions)
        epsilon = torch.randn(1) # epsilon ~ N(0, 1)
        z = mu + std * epsilon
        zs.append(z)

    m_loss = 0
    for i in range(B):
        states, actions =  subtrajectory_batch[i].actions, subtrajectory_batch[i].observations["observation"]
        mu_pi, std_pi = pi_theta(states, zs[i])
        pi_dist = Independent(Normal(mu_pi, std_pi), 1)
        log_pi_t = pi_dist.log_prob(actions)
        log_pi_seq = log_pi_t.sum(dim=1)

        mu_p_psi, std_p_psi = p_psi(states, zs[i])
        p_psi_dist = Independent(Normal(mu_p_psi, std_p_psi), 1)
        log_p_psi_t = p_psi_dist.log_prob(actions)
        log_p_psi_seq = log_p_psi_t.sum(dim=1)

        mu_p_omega, std_p_omega = p_omega(states, zs[i])
        p_omega_dist = Independent(Normal(mu_p_omega, std_p_omega), 1)
        log_p_omega_t = p_omega_dist.log_prob(actions)
        log_p_omega_seq = log_p_omega_t.sum(dim=1)  

        m_loss += log_pi_seq + log_p_psi_seq + log_p_omega_seq

    return m_loss * -(1/B)


EpisodeData(id=0, total_steps=1000, observations={achieved_goal: ndarray of shape (1001, 2) and dtype float64, desired_goal: ndarray of shape (1001, 2) and dtype float64, observation: ndarray of shape (1001, 27) and dtype float64}, actions=ndarray of shape (1000, 8) and dtype float32, rewards=ndarray of 1000 floats, terminations=ndarray of 1000 bools, truncations=ndarray of 1000 bools, infos=dict with the following keys: ['goal', 'qpos', 'qvel', 'success'])